# Getting Started: Atomic Systems Basic System Properties

Here, we present the basic properties of the LAMMPS-derived classes that will be later used for the implicit derivative calculations.

Currently, only the **linear-in-descriptor SNAP potentials** are used.

In [5]:
# Standard libraries
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

# No parallel MPI runs in the notebook, but keep comm for consistency
comm = None

### Import the package classes and functions

In [6]:
from lammps_implicit_der import SNAP
from lammps_implicit_der.systems import BCC, BCC_BINARY, BCC_BINARY_VACANCY, BCC_VACANCY, BCC_SIA, FromData, HCP
from lammps_implicit_der.tools import plot_tools

### Create a BCC system

As an instance of the `BCC` class. `BCC` is a child class of `LammpsImplicitDer`.

In [7]:
bcc_pure = BCC(alat=3.18, ncell_x=3, minimize=False,
               logname='bcc.log', lmp_cmd_filename='bcc_commands.lammps',
               snapcoeff_filename = 'W_REF.snapcoeff', verbose=True)


--------------------------------------------------------------------------------
Running LAMMPS with the following arguments:
-screen none -log bcc.log

Setting SNAP potential

                  SNAP coefficients for: W
                          quadraticflag: 0
 Number of parameters (excluding beta0): 55
                                Element:  W  |  R =  0.5000 w =  1.0000

Number of atoms: 54, largest force value: 2.098e-14, force norm: 1.323e-13


Here we used the following input arguments:

* `alat`: lattice parameter in Angstroms
* `ncell_x`: number of BCC unit cells in the x-direction. If `ncell_y` and `ncell_z` are not specified, they will be equal to `ncell_x`
* `minimize`: minimize the atomic positions
* `logname`: LAMMPS log file name
* `lmp_command_filename`: filename containing all the LAMMPS commands sent to LAMMPS for an object
* `snapcoeff_filename`: filename of the SNAP coefficients file. If `snapparam_filename` is not specified, it will be guessed by changing the `.snapcoeff` suffix to `.snapparam`

Refer to the docstring documentation of the `LammpsImplicitDer` class (`./lammps_implicit_der/lmp_der/implicit_der.py` file) and `BCC` child class (`./lammps_implicit_der/systems/bcc_lattices.py` file) for the full list of input parameters.

### Lammps object within the class

Under the hood, at the initialization stage, the `BCC` class creates a LAMMPS instance:

```python
self.lmp = lammps(cmdargs=self.cmdargs, comm=self.comm)
```

Working with `BCC` or any child class consists in sending commands and data to and receiving data from LAMMPS at runtime. All the commands sent to lammps will be stored in the `lmp_command_filename` file. The LAMMPS logs will be printed out in the `logname` file.

### Manually send a command to LAMMPS

In [8]:
bcc_pure.lmp_commands_string("run 0")

### Save the system to a LAMMPS data file

In [10]:
bcc_pure.write_data('bcc_pure.data')

# First 12 lines of the data file
with open('bcc_pure.data', 'r') as f:
    for i in range(12):
        print(f.readline().strip())
print('...')

LAMMPS data file via write_data, version 2 Aug 2023, timestep = 0, units = metal

54 atoms
1 atom types

0 9.540000000000001 xlo xhi
0 9.540000000000001 ylo yhi
0 9.540000000000001 zlo zhi

Masses

1 183.84
...


### Save the coordinates of the systems to a .xyz file

In [12]:
bcc_pure.write_xyz_file('bcc_pure.xyz')

# First 5 lines of the xyz file
with open('bcc_pure.xyz', 'r') as f:
    for i in range(5):
        print(f.readline().strip())
print('...')

54
Atomic coordinates generated by LammpsImplicitDer
W 0.031800 0.031800 0.031800
W 1.621800 1.621800 1.621800
W 3.211800 0.031800 0.031800
...


## Properties

### Energy

